In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY310,
)

In [2]:
import os
import re

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

from matplotlib.animation import FFMpegWriter, FuncAnimation
from matplotlib.colors import BoundaryNorm, ListedColormap

import lysis

pd.reset_option("display.precision")
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

In [3]:
experiment_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("file_code", np.str_, 40),
        ("forced_unbind", np.float_),
        ("average_bind_time", np.float_),
    ]
)
code_type = np.dtype([("file_code", np.str_, 40), ("descriptor", np.str_, 40), ("executable", np.str_, 40)])
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("experiment", np.str_, 40),
        ("code", np.str_, 40),
        ("seed", int),
        ("running_time", int),
    ]
)

In [4]:
experiments = np.array(
    [
        ("Physiological Kd", "", 8.52e-2, 27.8),
        ("10x bigger", "_Kd0236", 5.4e-3, 2.78),
        ("10x smaller", "_Kd00020036", 0.5143, 277.8),
    ],
    dtype=experiment_type,
)
programs = np.array(
    [
        ("_always", "Always bind", "macro_Q2_always_rebind"),
        ("_along", "Diffuse along clot", "macro_Q2_diffuse_along"),
        ("_into", "Diffuse into clot", "macro_Q2_diffuse_into"),
        # (
        #     "_into_and_along",
        #     "Diffuse into and along clot - BUGGED",
        #     "macro_Q2_diffuse_into_and_along",
        # ),
        (
            "_into_and_along_fixed",
            "Diffuse into and along clot",
            "macro_Q2_diffuse_into_and_along_fixed",
        ),
    ],
    dtype=code_type,
)
runs = np.empty(15, dtype=run_type)

In [5]:
in_file_code = "_PLG2_tPA01{data_code}_Q2.dat"
out_file_code = "_PLG2_tPA01{data_code}{program_code}_Q2.dat"

slope_tolerance = 1e-3
rng = np.random.default_rng(65463453)

In [6]:
group_code = "2023-02-02-22"
runs = np.array(
    [
        (group_code + "00", "Physiological Kd", "Diffuse along clot", 17109424, 1200),
        (group_code + "01", "Physiological Kd", "Always bind", 9965734, 1800),
        # (group_code + "02", "Physiological Kd", "Diffuse into and along clot - BUGGED", -2137354075, 1200),
        (group_code + "03", "Physiological Kd", "Diffuse into and along clot", -2137354075, 1800),
        (group_code + "04", "Physiological Kd", "Diffuse into clot", -2135977853, 1200),
        (group_code + "05", "10x smaller", "Diffuse along clot", -848304637, 1200),
        (group_code + "06", "10x smaller", "Always bind", 1299539472, 1800),
        # (group_code + "07", "10x smaller", "Diffuse into and along clot - BUGGED", -854989241, 1200),
        (group_code + "08", "10x smaller", "Diffuse into and along clot", -854989241, 1800),
        (group_code + "09", "10x smaller", "Diffuse into clot", -850336215, 1200),
        (group_code + "10", "10x bigger", "Diffuse along clot", -1216563743, 1200),
        (group_code + "11", "10x bigger", "Always bind", 669985532, 900),
        # (group_code + "12", "10x bigger", "Diffuse into and along clot - BUGGED", -1212172957, 1200),
        (group_code + "13", "10x bigger", "Diffuse into and along clot", -1212172957, 1800),
        (group_code + "14", "10x bigger", "Diffuse into clot", -1213352577, 1200),
    ],
    dtype=run_type,
)

In [7]:
index = pd.MultiIndex.from_product(
    [experiments["descriptor"], programs["descriptor"]], names=["data", "program"]
)
# index = [run['experiment'] + " - " + run['code'] for run in runs]
statistics = ["Mean front velocity", "Mean of Standard Deviation of front velocity"]
results = pd.DataFrame(index=index, columns=statistics)
# front_velocity_table = pd.DataFrame(index=programs["descriptor"], columns=experiments["descriptor"])

In [8]:
colormap_f = plt.cm.cividis(np.arange(plt.cm.cividis.N))
colormap_f[:, 3] = 0.3
colormap_f = ListedColormap(colormap_f)
colormap_f.set_extremes(bad="k", under="w", over="k")
colormap_m = ListedColormap(["green", "red"])
bounds = [0, 0.5, 1]
norm_m = BoundaryNorm(bounds, colormap_m.N)

In [9]:
def load_fortran_files(exp, file_code):
    n_save = np.fromfile(os.path.join(e.os_path, "Nsave" + file_code), dtype=np.int32)
    n_save += 1
    raw_deg = np.fromfile(os.path.join(e.os_path, "deg" + file_code))
    raw_tsave = np.fromfile(os.path.join(e.os_path, "tsave" + file_code))
    raw_mol_location = np.fromfile(os.path.join(e.os_path, "m_loc" + file_code), dtype=np.int32)
    raw_mol_status = np.fromfile(os.path.join(e.os_path, "m_bound" + file_code), dtype=np.int32)
    raw_mol_status = raw_mol_status.astype(np.bool_)
    raw_mfpt = np.fromfile(os.path.join(e.os_path, "mfpt" + file_code))

    mfpt = raw_mfpt.reshape(raw_mfpt.size // e.macro_params.total_molecules, e.macro_params.total_molecules)
    if mfpt.shape[0] > e.macro_params.total_trials:
        mfpt = mfpt[: e.macro_params.total_trials]
    # mfpt = mfpt[mfpt[:, -1] != 0]

    raw_mapped_deg = -raw_deg
    raw_mapped_deg[raw_deg == 0] = np.max(raw_tsave) + e.macro_params.save_interval  # float('inf') #
    raw_mapped_deg[raw_deg == -1] = 0

    mapped_deg = []
    tsave = []
    mol_location = []
    mol_status = []
    start = 0
    end = 0
    for r in range(e.macro_params.total_trials):
        start = end
        end += n_save[r]
        mapped_deg.append(
            raw_mapped_deg[start * e.macro_params.total_edges : end * e.macro_params.total_edges].reshape(
                n_save[r], e.macro_params.total_edges
            )
        )
        mol_location.append(
            raw_mol_location[
                start * e.macro_params.total_molecules : end * e.macro_params.total_molecules
            ].reshape(n_save[r], e.macro_params.total_molecules)
            - 1
        )
        mol_status.append(
            raw_mol_status[
                start * e.macro_params.total_molecules : end * e.macro_params.total_molecules
            ].reshape(n_save[r], e.macro_params.total_molecules)
        )
        tsave.append(raw_tsave[start:end])
    return n_save, mapped_deg, tsave, mfpt, mol_location, mol_status


def calculate_time_row_exposed(exp, deg):
    exposed_time = np.empty(
        (exp.macro_params.total_trials, exp.macro_params.rows - 1, exp.macro_params.cols), dtype=np.float_
    )
    for run in range(exp.macro_params.total_trials):
        for j in range(exp.macro_params.cols):
            for i in range(exp.macro_params.rows - 1):
                if i == 0:
                    exposed_time[run, i, j] = 0
                else:
                    k = lysis.to_fortran_edge_index(i, j, exp.macro_params.rows, exp.macro_params.cols)
                    exposed_time[run, i, j] = max(exposed_time[run, i - 1, j], deg[run][-1, k])
    # exposed_time = 10* np.ceil(exposed_time / 10)
    return exposed_time / 60


def find_degradation_fronts(exp, exposed_time, y_distance, tsave):
    deg_fronts = []
    for r in range(exp.macro_params.total_trials):
        run_deg_fronts = []
        for j in range(exp.macro_params.cols):
            col_deg_front = []
            for i in range(1, exp.macro_params.rows - 1):
                if exposed_time[r, i - 1, j] < exposed_time[r, i, j] < tsave[r][-1] + 1:
                    col_deg_front.append([exposed_time[r, i, j], y_distance[i]])
            run_deg_fronts.append(np.array(col_deg_front).T)
        deg_fronts.append(run_deg_fronts)
    return deg_fronts


# TODO(bpaynter): Change this later to do mean and std of all columns across all runs
def mean_front_velocity(exp, deg_fronts):
    run_mean_velocity = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    run_std_velocity = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    for run in range(exp.macro_params.total_trials):
        front_velocity = np.empty(exp.macro_params.cols, dtype=np.float_)
        for j in range(exp.macro_params.cols):
            b, m = np.polynomial.polynomial.polyfit(deg_fronts[run][j][0], deg_fronts[run][j][1], 1)
            front_velocity[j] = m
        run_mean_velocity[run] = np.mean(front_velocity)
        run_std_velocity[run] = np.std(front_velocity)
    return np.mean(run_mean_velocity), np.mean(run_std_velocity)


def plot_front_degradation(exp, file_code, deg_fronts, deg):
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_axis_on()
    ax.set_xlim(0, (max([np.max(deg_run[-1, :]) for deg_run in deg]) // 60) + 1)
    ax.set_ylim(
        (exp.macro_params.empty_rows - 1) * e.macro_params.grid_node_distance,
        (exp.macro_params.rows - 1) * exp.macro_params.grid_node_distance,
    )
    for run in range(exp.macro_params.total_trials):
        for j in range(exp.macro_params.cols):
            plt.plot(deg_fronts[run][j][0], deg_fronts[run][j][1], linewidth=1)
    fig.savefig(os.path.join(exp.os_path, "deg_fronts" + file_code[:-4] + ".png"), bbox_inches="tight")
    plt.close()


def find_degraded_percent(exp, deg, tsave):
    degraded_percent = []
    for r in range(exp.macro_params.total_trials):
        run_degraded_percent = np.empty(deg[r].shape[0], dtype=np.float_)
        for t in range(deg[r].shape[0]):
            run_degraded_percent[t] = np.count_nonzero(deg[r][t] <= tsave[r][t])
        run_degraded_percent -= exp.macro_params.empty_rows * exp.macro_params.full_row
        degraded_percent.append(run_degraded_percent / exp.macro_params.total_fibers)
    return degraded_percent


def mean_degradation_rate(exp, degraded_percent, tsave):
    degradation_rate = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    offset = np.empty(exp.macro_params.total_trials, dtype=np.float_)
    for r in range(exp.macro_params.total_trials):
        slope = np.empty(degraded_percent[r].shape[0], dtype=np.float_)
        slope[0] = degraded_percent[r][0]
        for t in range(1, degraded_percent[r].shape[0]):
            slope[t] = degraded_percent[r][t] - degraded_percent[r][t - 1]
        degradation_happening = slope_tolerance <= slope
        b, m = np.polynomial.polynomial.polyfit(
            tsave[r][degradation_happening] / 60, degraded_percent[r][degradation_happening], 1
        )
        degradation_rate[r] = m
        offset[r] = b
    return degradation_rate, offset


def plot_degradation_percent(exp, degraded_percent, tsave, degradation_rate, offset):
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_xlim(0, (max([np.max(deg_run[-1, :]) for deg_run in deg]) // 60) + 1)
    ax.set_ylim(-0.1, 1.1)
    for r in range(exp.macro_params.total_trials):
        plt.plot(tsave[r] / 60, degraded_percent[r])
        plt.plot(
            np.arange((max([np.max(deg_run[-1, :]) for deg_run in deg]) // 60) + 1) * degradation_rate[r]
            + offset[r],
            color="b",
            alpha=0.5,
            zorder=0.1,
        )
    fig.savefig(os.path.join(exp.os_path, "deg_rate" + file_code[:-4] + ".png"), bbox_inches="tight")
    plt.close()


def get_unbind_amounts(exp, file_code):
    macro_unbind_pattern = re.compile(r"countmacrounbd=\s*(\d+)")
    micro_unbind_pattern = re.compile(r"countmicrounbd=\s*(\d+)")
    log_file_name = os.path.join(exp.os_path, "macro" + file_code[:-4] + ".txt")
    with open(log_file_name, "r") as file:
        log_text = file.read()
    macro_unbinds = re.findall(macro_unbind_pattern, log_text)
    micro_unbinds = re.findall(micro_unbind_pattern, log_text)
    return np.array(macro_unbinds, dtype=int), np.array(micro_unbinds, dtype=int)


def plot_coords(i, j):
    x = j
    y = i
    if j % 3 == 0:
        return x / 3.0, y + 0.5
    if j % 3 == 1:
        return (x - 1) / 3.0, y
    if j % 3 == 2:
        return (x - 2) / 3.0 + 0.5, y


def get_edge_index(exp):
    edge_index = np.empty(exp.macro_params.total_edges, dtype=tuple)
    for k in range(exp.macro_params.total_edges):
        edge_index[k] = lysis.from_fortran_edge_index(k, exp.macro_params.rows, exp.macro_params.cols)
    return edge_index


def animation_data(exp, edge_index, mol_location):
    x_f = np.empty(exp.macro_params.total_edges, dtype=float)
    y_f = np.empty(exp.macro_params.total_edges, dtype=float)
    for k in range(exp.macro_params.total_edges):
        i, j = edge_index[k]
        x_f[k], y_f[k] = plot_coords(i, j)
    d_x = (rng.random(size=exp.macro_params.total_molecules) - 0.5) / 2.5
    d_y = (rng.random(size=exp.macro_params.total_molecules) - 0.5) / 2.5
    x_m = []
    y_m = []
    for r in range(exp.macro_params.total_trials):
        x = np.empty((n_save[r], exp.macro_params.total_molecules), dtype=np.float_)
        y = np.empty((n_save[r], exp.macro_params.total_molecules), dtype=np.float_)
        for t in range(n_save[r]):
            for k in range(exp.macro_params.total_molecules):
                i, j = edge_index[mol_location[r][t, k]]
                x[t, k], y[t, k] = plot_coords(i, j)
            x[t] += d_x
            y[t] += d_y
        x_m.append(x)
        y_m.append(y)
    return x_f, y_f, x_m, y_m


def create_animation(exp, file_code, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m):
    for run in range(1):
        fig = plt.figure(figsize=(121, 121), dpi=10)
        ax = fig.add_axes([0, 0, 1, 1])
        ax.set_axis_off()
        vmin = 0
        vmax = exp.macro_params.cols - 1

        ydiff = exp.macro_params.rows - exp.macro_params.cols

        ax.set_xlim(vmin - 0.25, vmax + 0.25)
        ax.set_ylim(vmin - 0.25, vmax + ydiff + 0.25)
        ax.set_aspect("equal")
        fig.canvas.draw()
        title = ax.annotate(f"t = {tsave[run][0]:.1f} sec", (0, 0), zorder=100)
        s = (ax.get_window_extent().width / (vmax - vmin + 0.5) * 72 / (2 * fig.dpi)) ** 2
        scatt_f = ax.scatter(
            x_f,
            y_f,
            s=s,
            marker="s",
            linewidths=0,
            c=deg[run][0],
            cmap=colormap_f,
            vmin=1,
            vmax=deg[run][-1][deg[run][-1] < max([max(t) for t in tsave])].max(),
        )
        scatt_m = ax.scatter(
            x_m[run][0],
            y_m[run][0],
            s=s / 2.5,
            marker="o",
            linewidths=0,
            c=mol_status[run][0],
            cmap=colormap_m,
            norm=norm_m,
        )

        def update(frame_number):
            scatt_f.set_array(deg[run][frame_number])
            scatt_m.set_array(mol_status[run][frame_number])
            scatt_m.set_offsets(np.append((x_m[run][frame_number],), (y_m[run][frame_number],), axis=0).T)
            title.set_text(f"t = {tsave[run][frame_number]:.1f} sec")

        animation = FuncAnimation(fig, update, frames=np.arange(n_save[run]), interval=200)

        FFwriter = FFMpegWriter(fps=10)
        animation.save(
            os.path.join(e.os_path, f"combined_animation_{run:02}" + file_code[:-4] + ".mp4"), writer=FFwriter
        )
        plt.close()


def create_animation_stills(exp, file_code, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m):
    for run in range(1):
        for time in [30, 60, 90, 120]:
            frame = int(np.argwhere(tsave[0] >= time)[0])

            fig = plt.figure(figsize=(12.1, 12.1), dpi=100)
            ax = fig.add_axes([0, 0, 1, 1])
            ax.set_axis_off()
            vmin = 0
            vmax = exp.macro_params.cols - 1

            ydiff = exp.macro_params.rows - exp.macro_params.cols

            ax.set_xlim(vmin - 0.25, vmax + 0.25)
            ax.set_ylim(vmin - 0.25, vmax + ydiff + 0.25)
            ax.set_aspect("equal")
            fig.canvas.draw()
            # title = ax.annotate(f"t = {tsave[run][frame]:.1f} sec", (0, 0), zorder=100)
            s = (ax.get_window_extent().width / (vmax - vmin + 0.5) * 72 / (2 * fig.dpi)) ** 2
            scatt_f = ax.scatter(
                x_f,
                y_f,
                s=s,
                marker="s",
                linewidths=0,
                c=deg[run][frame],
                cmap=colormap_f,
                vmin=1,
                vmax=deg[run][n_save[run] - 1][
                    deg[run][n_save[run] - 1] < max([max(t) for t in tsave])
                ].max(),
            )
            scatt_m = ax.scatter(
                x_m[run][frame],
                y_m[run][frame],
                s=s / 2.5,
                marker="o",
                linewidths=0,
                c=mol_status[run][frame],
                cmap=colormap_m,
                norm=norm_m,
            )

            fig.savefig(
                os.path.join(e.os_path, f"fiber_grid_plot_r{run:02}_t{time:03}" + file_code[:-4] + ".png"),
                bbox_inches="tight",
            )
            plt.close()

In [10]:
for run in runs:
    prog = programs[programs["descriptor"] == run["code"]]
    exper = experiments[experiments["descriptor"] == run["experiment"]]
    e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code=run["exp_code"])
    # e = lysis.util.Experiment(
    #     os.path.join("/", "home", "bpaynter", "Archive", "lysis_data"), experiment_code=run["exp_code"]
    # )
    e.read_file()
    y_distance = np.arange(e.macro_params.rows - 1) * e.macro_params.grid_node_distance
    file_code = out_file_code.format(data_code=exper["file_code"][0], program_code=prog["file_code"][0])
    print(run["exp_code"], file_code)
    n_save, deg, tsave, mfpt, mol_location, mol_status = load_fortran_files(e, file_code)
    exposed_time = calculate_time_row_exposed(e, deg)
    deg_fronts = find_degradation_fronts(e, exposed_time, y_distance, tsave)
    plot_front_degradation(e, file_code, deg_fronts, deg)
    m, sd = mean_front_velocity(e, deg_fronts)
    results.loc[(run["experiment"], run["code"]), "Mean front velocity"] = m
    results.loc[(run["experiment"], run["code"]), "Mean of Standard Deviation of front velocity"] = sd

    deg_percent = find_degraded_percent(e, deg, tsave)
    results.loc[(run["experiment"], run["code"]), "Mean degradation percent"] = (
        np.mean([run_deg_percent[-1] for run_deg_percent in deg_percent]) * 100
    )
    deg_rate, offset = mean_degradation_rate(e, deg_percent, tsave)
    results.loc[(run["experiment"], run["code"]), "Mean degradation rate"] = np.mean(deg_rate) * 100
    results.loc[(run["experiment"], run["code"]), "Standard deviation of degradation rate"] = (
        np.std(deg_rate) * 100
    )
    plot_degradation_percent(e, deg_percent, tsave, deg_rate, offset)

    results.loc[(run["experiment"], run["code"]), "Number of molecules that reached the back row"] = np.mean(
        np.count_nonzero(mfpt > 0, axis=1)
    )
    results.loc[(run["experiment"], run["code"]), "Percent of molecules that reached the back row"] = (
        np.mean(np.count_nonzero(mfpt > 0, axis=1)) / e.macro_params.total_molecules * 100
    )
    results.loc[(run["experiment"], run["code"]), "Mean first passage time (min)"] = np.mean(
        mfpt[mfpt > 0] / 60
    )
    results.loc[(run["experiment"], run["code"]), "Standard deviation of first passage time"] = np.std(
        mfpt[mfpt > 0] / 60
    )
    macro_unbinds, micro_unbinds = get_unbind_amounts(e, file_code)
    if macro_unbinds.size > 0:
        results.loc[(run["experiment"], run["code"]), "Mean number of macroscale unbinds"] = np.mean(
            macro_unbinds
        )
        results.loc[
            (run["experiment"], run["code"]), "Standard deviation in number of macroscale unbinds"
        ] = np.std(macro_unbinds)
    if micro_unbinds.size > 0:
        results.loc[(run["experiment"], run["code"]), "Mean number of microscale unbinds"] = np.mean(
            micro_unbinds
        )
        results.loc[
            (run["experiment"], run["code"]), "Standard deviation in number of microscale unbinds"
        ] = np.std(micro_unbinds)
    edge_index = get_edge_index(e)
    x_f, y_f, x_m, y_m = animation_data(e, edge_index, mol_location)
    create_animation(e, file_code, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m)
    create_animation_stills(e, file_code, edge_index, deg, mol_status, tsave, x_f, y_f, x_m, y_m)

2023-02-02-2200 _PLG2_tPA01_along_Q2.dat
2023-02-02-2201 _PLG2_tPA01_always_Q2.dat
2023-02-02-2203 _PLG2_tPA01_into_and_along_fixed_Q2.dat
2023-02-02-2204 _PLG2_tPA01_into_Q2.dat
2023-02-02-2205 _PLG2_tPA01_Kd00020036_along_Q2.dat
2023-02-02-2206 _PLG2_tPA01_Kd00020036_always_Q2.dat
2023-02-02-2208 _PLG2_tPA01_Kd00020036_into_and_along_fixed_Q2.dat
2023-02-02-2209 _PLG2_tPA01_Kd00020036_into_Q2.dat
2023-02-02-2210 _PLG2_tPA01_Kd0236_along_Q2.dat
2023-02-02-2211 _PLG2_tPA01_Kd0236_always_Q2.dat
2023-02-02-2213 _PLG2_tPA01_Kd0236_into_and_along_fixed_Q2.dat
2023-02-02-2214 _PLG2_tPA01_Kd0236_into_Q2.dat


In [11]:
results = results.astype({"Number of molecules that reached the back row": int})
results

Mean front velocity  \
data             program                                           
Physiological Kd Always bind                                6.78   
                 Diffuse along clot                         4.29   
                 Diffuse into clot                         11.00   
                 Diffuse into and along clot                6.25   
10x bigger       Always bind                               10.62   
                 Diffuse along clot                         9.45   
                 Diffuse into clot                         15.39   
                 Diffuse into and along clot               10.10   
10x smaller      Always bind                                6.25   
                 Diffuse along clot                         1.95   
                 Diffuse into clot                          3.07   
                 Diffuse into and along clot                2.47   

                                             Mean of Standard Deviation of front velocity  \
data             program                                                                    
Physiological Kd Always bind                                                         0.05   
                 Diffuse along clot                                                  0.03   
                 Diffuse into clot                                                   1.80   
                 Diffuse into and along clot                                         0.26   
10x bigger       Always bind                                                         0.16   
                 Diffuse along clot                                                  0.15   
                 Diffuse into clot                                                   0.59   
                 Diffuse into and along clot                                         0.18   
10x smaller      Always bind                                                         0.04   
                 Diffuse along clot                                                  0.02   
                 Diffuse into clot                                                   0.74   
                 Diffuse into and along clot                                         0.12   

                                              Mean degradation percent  \
data             program                                                 
Physiological Kd Always bind                                    100.00   
                 Diffuse along clot                             100.00   
                 Diffuse into clot                              100.00   
                 Diffuse into and along clot                    100.00   
10x bigger       Always bind                                    100.00   
                 Diffuse along clot                             100.00   
                 Diffuse into clot                              100.00   
                 Diffuse into and along clot                    100.00   
10x smaller      Always bind                                    100.00   
                 Diffuse along clot                             100.00   
                 Diffuse into clot                              100.00   
                 Diffuse into and along clot                    100.00   

                                              Mean degradation rate  \
data             program                                              
Physiological Kd Always bind                                   6.75   
                 Diffuse along clot                            4.24   
                 Diffuse into clot                            17.94   
                 Diffuse into and along clot                   7.91   
10x bigger       Always bind                                  10.57   
                 Diffuse along clot                            9.42   
                 Diffuse into clot                            16.07   
                 Diffuse into and along clot                  10.16   
10x smaller      Always bind              

In [12]:
e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code=runs[0]["exp_code"])
e.read_file()
results.to_json(os.path.join(e.os_path, "results.json"), orient="table")

In [13]:
compilations = {
    "Front Velocity": [
        "Mean front velocity",
        "Mean of Standard Deviation of front velocity",
    ],
    "Degradation Rate": ["Mean degradation rate", "Standard deviation of degradation rate"],
    "Mean First Passage Time": ["Mean first passage time (min)", "Standard deviation of first passage time"],
    "Macroscale Unbinds": [
        "Mean number of macroscale unbinds",
        "Standard deviation in number of macroscale unbinds",
    ],
    "Microscale Unbinds": [
        "Mean number of microscale unbinds",
        "Standard deviation in number of microscale unbinds",
    ],
}

In [14]:
which_display = "Degradation Rate"
display = pd.DataFrame(index=results.index)
display = results[compilations[which_display]].apply(lambda x: f"{x[0]:.2f} \u00B1 {x[1]:.2f}", axis=1)
display = display.unstack(0).reindex(results.index.get_level_values(1).unique())[
    results.index.get_level_values(0).unique()
]
print(display.style.to_latex())

\begin{tabular}{llll}
data & Physiological Kd & 10x bigger & 10x smaller \\
program &  &  &  \\
Always bind & 6.75 ± 0.01 & 10.57 ± 0.02 & 6.19 ± 0.01 \\
Diffuse along clot & 4.24 ± 0.01 & 9.42 ± 0.02 & 1.95 ± 0.01 \\
Diffuse into clot & 17.94 ± 0.14 & 16.07 ± 0.13 & 7.37 ± 0.03 \\
Diffuse into and along clot & 7.91 ± 0.05 & 10.16 ± 0.03 & 3.31 ± 0.03 \\
\end{tabular}



In [15]:
my_list = []
my_list[10] = 100
my_list

IndexError: list assignment index out of range

In [ ]:
mfpt.shape

In [ ]:
np.count_nonzero(mfpt > 0, axis=1)